# 3. Programming or SQL

Librerías

In [81]:
import pandas as pd
import numpy as np

Proceso

In [20]:
import csv

# Nombre del archivo CSV
archivo_csv = r"G:/Mi unidad/Prueba Bancolombia/RUAF.csv"

# Lista para almacenar todas las filas del archivo CSV
datos = []

with open(archivo_csv, newline='', encoding='latin-1"') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        # Verificar si la fila contiene el patrón "¬-*-¬"
        if "¬-*-¬" not in ','.join(row):  # Unimos los elementos de la fila en una cadena para buscar el patrón
            datos.append(row)

datos


[['5|CC;1116912531;VINCULACIÓN A PROGRAMAS DE  ASISTENCIA SOCIAL;;2021-09-09 17:14:36'],
 ['5|CC;1116912531;PENSIONADOS;;2021-09-09 17:14:36'],
 ['5|CC;1116912531;AFILIACIÓN A CESANTIAS;;2021-09-09 17:14:36'],
 ['5|CC;1116912531;AFILIACIÓN A RIESGOS LABORALES;;2021-09-09 17:14:36'],
 ['5|CC;1116912531;INFORMACIÓN BASICA;Segundo Apellido;M;2021-09-09 17:14:36'],
 ['5|CC;1116912531;INFORMACIÓN BASICA;Primer Apellido;BEDOYA;2021-09-09 17:14:36'],
 ['5|CC;1116912531;INFORMACIÓN BASICA;Segundo Nombre;MEJIA;2021-09-09 17:14:36'],
 ['5|CC;1116912531;INFORMACIÓN BASICA;Primer Nombre;MAURICIO;2021-09-09 17:14:36'],
 ['5|CC;1116912531;INFORMACIÓN BASICA;Número de Identificación;CC 1116912531;2021-09-09 17:14:36'],
 ['5|CC;1116912531;AFILIACIÓN A SALUD;Departamento -> Municipio;FLORENCIA;2021-09-09 17:14:36'],
 ['5|CC;1116912531;AFILIACIÓN A SALUD;Tipo de Afiliado;COTIZANTE;2021-09-09 17:14:36'],
 ['5|CC;1116912531;AFILIACIÓN A SALUD;Estado de Afiliación;Activo;2021-09-09 17:14:36'],
 ['5|CC;1116

In [75]:
# Convertir a df
df = pd.DataFrame([x[0].split(';') for x in datos], columns=["tipo_doc", "num_doc", "seccion", "campo", "valor_1","valor_2","valor_3"])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27338 entries, 0 to 27337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tipo_doc  27338 non-null  object
 1   num_doc   27338 non-null  object
 2   seccion   27338 non-null  object
 3   campo     27338 non-null  object
 4   valor_1   27241 non-null  object
 5   valor_2   23247 non-null  object
 6   valor_3   3 non-null      object
dtypes: object(7)
memory usage: 1.5+ MB


In [76]:
# Eliminar columnas innecesarias
df = df.drop(columns=['seccion','valor_2','valor_3'])
# Generar pivoteo del df
df_2 = df.pivot_table(index=['tipo_doc', 'num_doc'], columns=['campo'], aggfunc= lambda x: x)
# Fusionar los niveles de columnas
df_2.columns = df_2.columns.map('_'.join)
# Quedarse solo con la segunda fila de encabezados de columnas
df_2.columns = df_2.columns.str.split('_').str[2]

In [77]:
# Se integran las columnas del index
df_2 = df_2.reset_index()
df_2.columns

Index(['tipo_doc', 'num_doc', '', 'Actividad Economica', 'Administradora',
       'Administradora CF', 'Departamento -> Municipio',
       'Entidad que reconoce la pensión', 'Estado', 'Estado de Afiliación',
       'Estado de la Vinculación', 'Estado del Beneficio', 'Fecha Afiliacion',
       'Fecha Resolución', 'Fecha Ultimo Beneficio', 'Fecha de Afiliación',
       'Fecha de Vinculación', 'Modalidad', 'Municipio Labora',
       'Número Resoluciòn Pension PG', 'Número de Identificación',
       'Primer Apellido', 'Primer Nombre', 'Programa', 'Régimen',
       'Segundo Apellido', 'Segundo Nombre', 'Sexo', 'Tipo de Afiliado',
       'Tipo de Miembro de la Población Cubierta', 'Tipo de Pensionado',
       'Tipo de Pensión', 'Ubicación de Entrega del Beneficio'],
      dtype='object')

In [78]:
# Se elimina columna vacía
df_2 = df_2.drop(columns=[''])

In [79]:
# Convertir nombres de columnas a snake_case
df_2.columns = df_2.columns.str.lower()  # Convertir a minúsculas primero
df_2.columns = df_2.columns.str.replace(' ', '_')  # Reemplazar espacios por guiones bajos
# Convertir datos a snake_case
df_2 = df_2.map(lambda x: str(x).lower().replace(' ', '_') if isinstance(x, str) else x)

In [95]:
#Verificar si un valor es nulo o vacío

for x in df_2[['actividad_economica', 'administradora',
    'administradora_cf', 'departamento_->_municipio',
    'entidad_que_reconoce_la_pensión', 'estado', 'estado_de_afiliación',
    'estado_de_la_vinculación', 'estado_del_beneficio', 'fecha_afiliacion',
    'fecha_resolución', 'fecha_ultimo_beneficio', 'fecha_de_afiliación',
    'fecha_de_vinculación', 'modalidad', 'municipio_labora',
    'número_resoluciòn_pension_pg', 'número_de_identificación',
    'primer_apellido', 'primer_nombre', 'programa', 'régimen',
    'segundo_apellido', 'segundo_nombre', 'sexo', 'tipo_de_afiliado',
    'tipo_de_miembro_de_la_población_cubierta', 'tipo_de_pensionado',
    'tipo_de_pensión', 'ubicación_de_entrega_del_beneficio']].iterrows():
    if pd.isnull(x) or x == '':
        df_2['marca_sin_informacion'] = 1
    else:
        df_2['marca_sin_informacion'] = 0

In [98]:
#Se exportan los datos
df_2.to_csv("G:\Mi unidad\Prueba Bancolombia\RUAF_TRANSFORMADO.csv", index=False)